In [ ]:
import numpy as np
import tensorflow as tf

# Para reproducibilidad
tf.random.set_seed(0)
np.random.seed(0)

# -----------------------------
# Datos (4 clientes)
# -----------------------------
X = np.array([
    [ 1.0, 0.0, 1.0],
    [ 0.0, 1.0, 1.0],
    [ 1.0, 1.0, 0.0],
    [-1.0, 1.0, 0.5],
], dtype=np.float32)

y = np.array([
    [ 0.5],
    [ 0.0],
    [ 0.4],
    [-0.3],
], dtype=np.float32)

# -----------------------------
# Parámetros iniciales dados
# (OJO: en Keras Dense kernel tiene forma (input_dim, units))
# Tú diste W1 de forma (4,3). Entonces en Keras usamos W1.T de forma (3,4).
# -----------------------------
W1 = np.array([
    [ 0.2, -0.1,  0.0],
    [-0.3,  0.1,  0.2],
    [ 0.0,  0.2, -0.2],
    [ 0.1,  0.0,  0.3],
], dtype=np.float32)

b1 = np.array([0.0, 0.1, -0.1, 0.05], dtype=np.float32)

Wout = np.array([[ 0.1, -0.2, 0.0, 0.3]], dtype=np.float32)
bout = np.array([0.05], dtype=np.float32)

# -----------------------------
# Modelo 3-4-1
# -----------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(3,)),
    tf.keras.layers.Dense(4, activation="tanh", use_bias=True, name="hidden"),
    tf.keras.layers.Dense(1, activation="linear", use_bias=True, name="out"),
])

# Setear pesos manualmente
hidden = model.get_layer("hidden")
out = model.get_layer("out")

hidden.set_weights([W1.T, b1])         # kernel (3,4), bias (4,)
out.set_weights([Wout.T, bout])        # kernel (4,1), bias (1,)

# Pérdida: (1/2)*MSE para coincidir con (y - yhat)^2 / 2
def half_mse(y_true, y_pred):
    return 0.5 * tf.reduce_mean(tf.square(y_true - y_pred))

opt = tf.keras.optimizers.SGD(learning_rate=0.1)

model.compile(optimizer=opt, loss=half_mse)

# -----------------------------
# Callback para imprimir cada época
# -----------------------------
class PrintEpoch(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        yhat = self.model.predict(X, verbose=0).reshape(-1)

        W1_k, b1_k = self.model.get_layer("hidden").get_weights()
        Wout_k, bout_k = self.model.get_layer("out").get_weights()

        print("\n" + "="*60)
        print(f"Época {epoch+1}/5")
        
        # Mostrar pesos como en tu notación (W1 como (4,3) y Wout como (1,4))
        print("\nW1 (4x3):\n", np.array2string(W1_k.T, precision=9, separator=", "))
        print("b1 (4,):\n", np.array2string(b1_k, precision=9, separator=", "))
        print("\nWout (1x4):\n", np.array2string(Wout_k.T, precision=9, separator=", "))
        print("bout (1,):\n", np.array2string(bout_k, precision=9, separator=", "))
        print(f"\n##############    RESULTADOS    ##############")
        print(f"\nPérdida (L_prom): {logs['loss']:.12f}")
        print("y_hat:", np.array2string(yhat, precision=9, separator=", "))

# -----------------------------
# Entrenar 5 épocas 
# -----------------------------

B = 4 # 4 para gradiente descendente; 1 para gradiente estocástico; 1< y <4 para minibatch de tamaño B

model.fit(X, y, epochs=5, batch_size=B, verbose=0, callbacks=[PrintEpoch()])